In [6]:
import sys

In [43]:
# !{sys.executable} -m pip  install langchain_experimental langchain_core
# !{sys.executable} -m pip  install google-generativeai==0.3.1
# !{sys.executable} -m pip  install google-ai-generativelanguage==0.4.0
# !{sys.executable} -m pip  install langchain-google-genai
# !{sys.executable} -m pip  install "langchain[docarray]"
# !{sys.executable} -m pip install -U docarray
# !{sys.executable} -m pip install google-cloud-aiplatform[reasoningengine,langchain]
# !{sys.executable} -m pip install -U langchain-google-vertexai

In [1]:
import vertexai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

from vertexai.preview.generative_models import GenerativeModel, ChatSession 
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
)
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import VertexAI
from langchain.schema.runnable import RunnableMap
from langchain.utilities import GoogleSearchAPIWrapper
import pprint

project_id = "project_id"
location = "project_id"
vertexai.init(project=project_id, location=location)

api_key = "KEY"
cse_key = "cXXXX"
cse_key_restricted = "cXXX"

import pandas as pd

In [2]:

search = GoogleSearchAPIWrapper(google_api_key=api_key,google_cse_id=cse_key)
search_restricted = GoogleSearchAPIWrapper(google_api_key=api_key,google_cse_id=cse_key_restricted, siterestrict=True)

model  = GenerativeModel("gemini-1.0-pro")

llm_vertex = VertexAI(
    model_name="gemini-1.0-pro",
    max_output_tokens=756,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)



template = """Generate company description of company_name, based only on the following context and follow the given instruction:
{context}

Instruction : The description should contain information about when compnay started , what it does and what type of industry it is.
also dont be baised about that if its a good or great .just be neutral and dont add any firmographic information

Return you answer in short description of company 

company_name: {company_name}
"""

# template =  """Answer the question a a full sentence, based only on the following context:
# {context}

# Return you answer in three back ticks

# Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()


/home/ubuntu/setup_ray/.ray_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


In [4]:

def extract_defination(comp_name):
    
    results = search.results(comp_name,num_results=10)
    
    embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@003")
    
    text_list = pd.DataFrame(results)['snippet'].to_list()
    vectorstore = DocArrayInMemorySearch.from_texts(
        # mini docs for embedding
        text_list,
        embedding=embeddings # passing in the embedder model
    )
    
    retriever = vectorstore.as_retriever()


    chain = RunnableMap({
        "context": lambda x: retriever.get_relevant_documents(x["company_name"]),
        "company_name": lambda x: x["company_name"]
    }) | prompt | llm_vertex | output_parser
    
    defination= chain.invoke({"company_name": comp_name})

    return defination

In [5]:
extract_defination(comp_name='AvayaLLC')

/home/ubuntu/setup_ray/.ray_env/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
/home/ubuntu/setup_ray/.ray_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"## Avaya LLC: \n\nAvaya LLC is an American multinational technology company headquartered in Morristown, New Jersey. The company provides a range of communication and collaboration solutions, including cloud-based and on-premises systems. Avaya was founded in 2000 through a merger of Lucent Technologies' Enterprise Networks division and Canada's Northern Telecom. \n\n**Industry:** Technology \n\n**Founded:** 2000 \n\n**Services:** Communication and collaboration solutions \n"

In [7]:

compnay_list = companies = ['Trane Inc.'
,'Avaya LLC'
,'Ceridian HCM Inc.'
,'Keystone Automotive Industries, Inc.'
,'Puget Sound Energy Inc'
,'Sequa Corporation'
,'JBS USA Holdings, Inc.'
,'Capital Group Companies, Inc.'
,'Battelle Memorial Institute Inc.'
,"Schwan's Company"
,'American Foods Group LLC'
,'United Van Lines, LLC'
,'Breakthru Beverage Group LLC'
,'MetroGoldwynMayer Inc.']


df_compnaydef = pd.DataFrame(compnay_list,columns=['compnay_names'])

In [8]:
df_compnaydef['defination'] = df_compnaydef['compnay_names'].apply(lambda x : extract_defination(x) )

In [9]:
df_compnaydef.values[4]

array(['Puget Sound Energy Inc',
       "## Puget Sound Energy Inc.\n\nPuget Sound Energy (PSE) is an energy utility company based in Bellevue, Washington. Founded in 1886, PSE provides electricity and natural gas to over 1.1 million customers in the Puget Sound region of Washington state. As a regulated utility, PSE operates under the oversight of the Washington Utilities and Transportation Commission. \n\nThe company's core business is providing safe, reliable, and affordable energy to its customers. PSE also invests in renewable energy sources and energy efficiency programs to help reduce its environmental impact. \n"],
      dtype=object)

In [10]:
df_compnaydef.to_csv('compnay_description.csv',index=False)